## 1. Analysis of costs to run animal shelters
<p>You are working as a data analyst in the animal welfare department. In preparation for next year's budget, the head of your department would like to know the total cost to shelter animals, broken down by each animal type and size combination (i.e., 'small dogs', 'large dogs', etc.). You need to prepare a table that contains this information.</p>
<p>The total cost to shelter an animal for the year is calculated as the sum of three factors: the size and animal type, the age, and the location.</p>
<ul>
<li>The base cost of sheltering an animal is based upon its size and type. The costs per animal type and size are contained in the <code>size_costs</code> table. The criteria for classifying size has recently been updated, and so you will need to categorize animals based upon the following table:</li>
</ul>
<table>
<thead>
<tr>
<th style="text-align:right;"></th>
<th style="text-align:right;">Small</th>
<th>Medium</th>
<th>Large</th>
</tr>
</thead>
<tbody>
<tr>
<td style="text-align:right;">Dog</td>
<td style="text-align:right;">&lt;= 10 lbs</td>
<td>10 lbs &lt; and &lt;= 30 lbs</td>
<td>30 lbs &lt;</td>
</tr>
<tr>
<td style="text-align:right;">Cat</td>
<td style="text-align:right;">&lt;= 5 lbs</td>
<td>5 lbs &lt; and &lt;= 7 lbs</td>
<td>7 lbs &lt;</td>
</tr>
<tr>
<td style="text-align:right;">Bird</td>
<td style="text-align:right;">&lt;= 0.7 lbs</td>
<td>0.7 lbs &lt; and &lt;= 1.1 lbs</td>
<td>1.1 lbs &lt;</td>
</tr>
</tbody>
</table>
<ul>
<li>Older animals cost more, and so an age cost (contained in the <code>age_costs</code> table) is added. Each animal's age should be calculated as the age by the end of the year (December 31st, 2021).</li>
<li>There is a location cost depending on where the animal is sheltered (contained in the <code>location_costs</code> table). </li>
<li>The calculation should not include animals that have been sponsored by private charities (sponsored animals are listed in the <code>sponsored_animals</code> table).</li>
</ul>
<p>For future visualization purposes, you will also need to include a <code>percentage</code> column in your result. This percentage should reflect the fraction of the total cost to be allocated to each animal and size combination.</p>
<p>The data you need is available in the tables shown in the database schema below.</p>
<h5 id="databaseschema">Database Schema</h5>
<p><img src="https://assets.datacamp.com/production/repositories/5934/datasets/a946a159c0024ee0995f7a030f2c0cf802203835/diagram.PNG" width="500" height="500"> </p>

In [70]:
%%sql
postgresql:///animal_shelters

    select animalid,animaltype,
    extract(year from age(date('2021-12-31'),date(birthdate))) as age,
    case when animaltype='Dog' and weight<=10 then 'DS'
    when animaltype='Dog' and weight<=30 then 'DM'
    when animaltype='Dog' and weight>30 then 'DL'
    when animaltype='Cat' and weight<=5 then 'CS'
    when animaltype='Cat' and weight<=7 then 'CM'
    when animaltype='Cat' and weight>7 then 'CL'
    when animaltype='Bird' and weight<=0.7 then 'BS'
    when animaltype='Bird' and weight<=1.1 then 'BM'
    else 'BL' end as sizeid
    
    from animals as a
    left join location_costs as l
    on a.location=l.location
  
  
    where animalid not in (select sponsorid from sponsored_animals)
    limit 5

5 rows affected.


animalid,animaltype,age,sizeid
968bf2e7230d2f,Cat,9.0,CM
8aacfe19c56b0f,Dog,8.0,DL
9617240ad5b268,Cat,7.0,CL
fd7bf1fa986957,Dog,4.0,DL
691081c83b1075,Dog,5.0,DS


In [71]:
%%sql
postgresql:///animal_shelters
    

'Connected: @animal_shelters'

In [72]:
%%sql
postgresql:///animal_shelters
    with full_table as (
    with animals as(
    select *,
    cast((extract(year from age(date('2021-12-31'),
                                date(birthdate)))) as integer) as age,
    case when animaltype='Dog' and weight<=10 then 'DS'
    when animaltype='Dog' and weight<=30 then 'DM'
    when animaltype='Dog' and weight>30 then 'DL'
    when animaltype='Cat' and weight<=5 then 'CS'
    when animaltype='Cat' and weight<=7 then 'CM'
    when animaltype='Cat' and weight>7 then 'CL'
    when animaltype='Bird' and weight<=0.7 then 'BS'
    when animaltype='Bird' and weight<=1.1 then 'BM'
    else 'BL' end as sizeid
    from animals as a),
    
    age_costs as (
    select cast(age as integer) as age,
        cast(costs as integer) as cost
    from age_costs
    ),
    
    size_costs as (
    select sizeid,
        size,
        cast(costs as integer) as scosts
    from size_costs
    ),
    
    location_costs as (
    select location,
        cast(costs as integer) as lcosts
    from location_costs
    )
    
    
    
    
    select *,
    (lcosts+scosts+cost) as total
    from animals
    left join location_costs as l
    on animals.location=l.location
    left join age_costs
    on animals.age=age_costs.age
    left join size_costs as s
    on animals.sizeid=s.sizeid)
    
    select animaltype,size,
    sum(total) as total,
    sum(full_table.total)
    from full_table
    group by animaltype,size
    

9 rows affected.


animaltype,size,total,sum
Bird,Large,9325,9325
Bird,Medium,4345,4345
Bird,Small,1825,1825
Cat,Large,540295,540295
Cat,Medium,305130,305130
Cat,Small,631130,631130
Dog,Large,1198655,1198655
Dog,Medium,1143435,1143435
Dog,Small,405770,405770


In [73]:
%%sql
postgresql:///animal_shelters
    with full_table as (
    with animals as(
    select *,
    cast((extract(year from age(date('2021-12-31'),
                                date(birthdate)))) as integer) as age,
    case when animaltype='Dog' and weight<=10 then 'DS'
    when animaltype='Dog' and weight<=30 then 'DM'
    when animaltype='Dog' and weight>30 then 'DL'
    when animaltype='Cat' and weight<=5 then 'CS'
    when animaltype='Cat' and weight<=7 then 'CM'
    when animaltype='Cat' and weight>7 then 'CL'
    when animaltype='Bird' and weight<=0.7 then 'BS'
    when animaltype='Bird' and weight<=1.1 then 'BM'
    else 'BL' end as sizeid
    from animals as a),
    
    age_costs as (
    select cast(age as integer) as age,
        cast(costs as integer) as cost
    from age_costs
    ),
    
    size_costs as (
    select sizeid,
        size,
        cast(costs as integer) as scosts
    from size_costs
    ),
    
    location_costs as (
    select location,
        cast(costs as integer) as lcosts
    from location_costs
    )
    
    
    
    
    select *,
    (lcosts+scosts+cost) as total
    from animals
    left join location_costs as l
    on animals.location=l.location
    left join age_costs
    on animals.age=age_costs.age
    left join size_costs as s
    on animals.sizeid=s.sizeid),
    
    total_cost as(
        with ddd as (select *,
    extract(year from age(date('2021-12-31'),date(birthdate))) as age,
    case when animaltype='Dog' and weight<=10 then 'DS'
    when animaltype='Dog' and weight<=30 then 'DM'
    when animaltype='Dog' and weight>30 then 'DL'
    when animaltype='Cat' and weight<=5 then 'CS'
    when animaltype='Cat' and weight<=7 then 'CM'
    when animaltype='Cat' and weight>7 then 'CL'
    when animaltype='Bird' and weight<=0.7 then 'BS'
    when animaltype='Bird' and weight<=1.1 then 'BM'
    else 'BL' end as sizeid
                 
    from animals as a
    left join location_costs as l
    on a.location=l.location
  
  
    where animalid not in (select sponsorid from sponsored_animals))
    
    select sum(costs)
    from ddd)
    
    select animaltype,size,
    sum(total) as total
    from full_table
    group by animaltype,size
    

9 rows affected.


animaltype,size,total
Cat,Small,631130
Cat,Large,540295
Cat,Medium,305130
Dog,Small,405770
Bird,Small,1825
Dog,Large,1198655
Bird,Medium,4345
Bird,Large,9325
Dog,Medium,1143435


In [74]:
%%sql
postgresql:///animal_shelters
    
    select animaltype, size, total,total/100
    from    
    (with full_table as (
    with animals as(
    select *,
    cast((extract(year from age(date('2021-12-31'),
                                date(birthdate)))) as integer) as age,
    case when animaltype='Dog' and weight<=10 then 'DS'
    when animaltype='Dog' and weight<=30 then 'DM'
    when animaltype='Dog' and weight>30 then 'DL'
    when animaltype='Cat' and weight<=5 then 'CS'
    when animaltype='Cat' and weight<=7 then 'CM'
    when animaltype='Cat' and weight>7 then 'CL'
    when animaltype='Bird' and weight<=0.7 then 'BS'
    when animaltype='Bird' and weight<=1.1 then 'BM'
    else 'BL' end as sizeid
    from animals as a),
    
    age_costs as (
    select cast(age as integer) as age,
        cast(costs as integer) as cost
    from age_costs
    ),
    
    size_costs as (
    select sizeid,
        size,
        cast(costs as integer) as scosts
    from size_costs
    ),
    
    location_costs as (
    select location,
        cast(costs as integer) as lcosts
    from location_costs
    )
    
    
    
    
    select *,
    (lcosts+scosts+cost) as total
    from animals
    left join location_costs as l
    on animals.location=l.location
    left join age_costs
    on animals.age=age_costs.age
    left join size_costs as s
    on animals.sizeid=s.sizeid)
    
    select animaltype,size,
    sum(total) as total
    from full_table
    group by animaltype,size)
    

(psycopg2.errors.SyntaxError) subquery in FROM must have an alias
LINE 3:     (with full_table as (
            ^
HINT:  For example, FROM (SELECT ...) [AS] foo.

[SQL: select animaltype, size, total,total/100
    from    
    (with full_table as (
    with animals as(
    select *,
    cast((extract(year from age(date('2021-12-31'),
                                date(birthdate)))) as integer) as age,
    case when animaltype='Dog' and weight<=10 then 'DS'
    when animaltype='Dog' and weight<=30 then 'DM'
    when animaltype='Dog' and weight>30 then 'DL'
    when animaltype='Cat' and weight<=5 then 'CS'
    when animaltype='Cat' and weight<=7 then 'CM'
    when animaltype='Cat' and weight>7 then 'CL'
    when animaltype='Bird' and weight<=0.7 then 'BS'
    when animaltype='Bird' and weight<=1.1 then 'BM'
    else 'BL' end as sizeid
    from animals as a),
    
    age_costs as (
    select cast(age as integer) as age,
        cast(costs as integer) as cost
    from age_costs
    ),


In [75]:
%%sql
postgresql:///animal_shelters
    
    select *
    from size_costs

9 rows affected.


sizeid,animaltype,size,costs
DL,Dog,Large,175
DM,Dog,Medium,150
DS,Dog,Small,125
CL,Cat,Large,140
CM,Cat,Medium,120
CS,Cat,Small,100
BL,Bird,Large,110
BM,Bird,Medium,90
BS,Bird,Small,70


In [76]:
%%sql
postgresql:///animal_shelters
    with animals as(
    select *,
    cast((extract(year from age(date('2021-12-31'),
                                date(birthdate)))) as integer) as age,
    case when animaltype='Dog' and weight<=10 then 'DS'
    when animaltype='Dog' and weight<=30 then 'DM'
    when animaltype='Dog' and weight>30 then 'DL'
    when animaltype='Cat' and weight<=5 then 'CS'
    when animaltype='Cat' and weight<=7 then 'CM'
    when animaltype='Cat' and weight>7 then 'CL'
    when animaltype='Bird' and weight<=0.7 then 'BS'
    when animaltype='Bird' and weight<=1.1 then 'BM'
    else 'BL' end as sizeid
    from animals as a),
    
    age_costs as (
    select cast(age as integer) as age,
        cast(costs as integer) as cost
    from age_costs
    ),
    
    size_costs as (
    select sizeid,
        cast(costs as integer) as scosts
    from size_costs
    ),
    
    location_costs as (
    select location,
        cast(costs as integer) as lcosts
    from location_costs
    )
    
    
    
    
    select *
    from animals
    
    left join location_costs as l
    on animals.location=l.location
    left join age_costs
    on animals.age=age_costs.age
    left join size_costs as s
    on animals.sizeid=s.sizeid
  
    
    limit 5

5 rows affected.


animalid,birthdate,animaltype,color,weight,location,age,sizeid,location_1,lcosts,age_1,cost,sizeid_1,scosts
80066a4b41ac06,05/11/2019,Cat,Blue Tabby/White,4.0,Fort Worth,2,CS,Fort Worth,100,2,0,CS,100
968bf2e7230d2f,11/12/2012,Cat,Torbie,7.0,Houston,9,CM,Houston,140,9,150,CM,120
8aacfe19c56b0f,07/08/2013,Dog,White/Tan,45.0,Austin,8,DL,Austin,135,8,150,DL,175
7126d96889803f,04/05/2015,Cat,Black,3.0,Houston,6,CS,Houston,140,6,100,CS,100
9617240ad5b268,04/29/2014,Cat,Black,9.0,Houston,7,CL,Houston,140,7,100,CL,140


# START OVER

In [77]:
%%sql
postgresql:///animal_shelters
    select animalid,table1.animaltype,size,
            (age_costs.costs+location_costs.costs+size_costs.costs) as total
            from (select animalid,animaltype,location,
                cast((extract(year from age(date('2021-12-31'),
                                            date(birthdate)))) as integer) as age,
                case when animaltype='Dog' and weight<=10 then 'DS'
                when animaltype='Dog' and weight<=30 then 'DM'
                when animaltype='Dog' and weight>30 then 'DL'
                when animaltype='Cat' and weight<=5 then 'CS'
                when animaltype='Cat' and weight<=7 then 'CM'
                when animaltype='Cat' and weight>7 then 'CL'
                when animaltype='Bird' and weight<=0.7 then 'BS'
                when animaltype='Bird' and weight<=1.1 then 'BM'
                else 'BL' end as sizeid
                from animals) 
            as table1
            left join location_costs on location_costs.location=table1.location
            left join size_costs on size_costs.sizeid=table1.sizeid
            left join (select cast(age as integer),costs from age_costs) as age_costs
            on age_costs.age=table1.age
            where animalid not in (select sponsorid from sponsored_animals)
            
            

9757 rows affected.


animalid,animaltype,size,total
968bf2e7230d2f,Cat,Medium,410
8aacfe19c56b0f,Dog,Large,460
9617240ad5b268,Cat,Large,380
fd7bf1fa986957,Dog,Large,310
691081c83b1075,Dog,Small,325
f957d0fabf9e63,Dog,Medium,250
2f17d474be934d,Dog,Medium,285
e03a24b436f6db,Dog,Medium,440
fd8befa64f05ff,Cat,Large,280
b8f7754e09af92,Cat,Small,235


In [78]:
%%sql
postgresql:///animal_shelters
    
    
    select lower(animaltype) as animaltype,lower(size) as size,total, round(total*100/sum(total) over(),2) as percentage
    from(    
    
        select animaltype,size,sum(total) as total
        from(

            select animalid,table1.animaltype,size,
            (age_costs.costs+location_costs.costs+size_costs.costs) as total
            from (select animalid,animaltype,location,
                cast((extract(year from age(date('2021-12-31'),
                                            date(birthdate)))) as integer) as age,
                case when animaltype='Dog' and weight<=10 then 'DS'
                when animaltype='Dog' and weight<=30 then 'DM'
                when animaltype='Dog' and weight>30 then 'DL'
                when animaltype='Cat' and weight<=5 then 'CS'
                when animaltype='Cat' and weight<=7 then 'CM'
                when animaltype='Cat' and weight>7 then 'CL'
                when animaltype='Bird' and weight<=0.7 then 'BS'
                when animaltype='Bird' and weight<=1.1 then 'BM'
                else 'BL' end as sizeid
                from animals) 
            as table1
            left join location_costs on location_costs.location=table1.location
            left join size_costs on size_costs.sizeid=table1.sizeid
            left join (select cast(age as integer),costs from age_costs) as age_costs
            on age_costs.age=table1.age
            where animalid not in (select sponsorid from sponsored_animals)
        )
        as table2
        group by animaltype,size) as table3
    order by animaltype,size desc
    
    
    
   




9 rows affected.


animaltype,size,total,percentage
bird,small,1615,0.05
bird,medium,3460,0.10
bird,large,7770,0.22
cat,small,518015,14.90
cat,medium,250575,7.21
cat,large,439490,12.64
dog,small,336530,9.68
dog,medium,941895,27.09
dog,large,977665,28.12
